In [3]:
import numpy as np
import os
import cPickle as pkl
import scipy.signal
import numpy.fft as fft
import sys
import optparse
from PIL import Image
import re
import itertools
from scipy import ndimage

import time
import datetime

#import pandas as pd

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure
import csv

import pymworks 
#import pandas as pd
import operator
import codecs

import scipy.io
import copy


# Abstract struct class       
class Struct:
    def __init__ (self, *argv, **argd):
        if len(argd):
            # Update by dictionary
            self.__dict__.update (argd)
        else:
            # Update by position
            attrs = filter (lambda x: x[0:2] != "__", dir(self))
            for n in range(len(argv)):
                setattr(self, attrs[n], argv[n])


class cycstruct(Struct):
    times = []
    idxs = 0
    vals = 0
    ordernum = 0
    triggers = 0

def get_timekey(item):
    return item.time


In [4]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
def get_session_bounds(dfn):
    df = None
    df = pymworks.open(dfn)                                                          # Open the datafile

    # First, find experiment start, stop, or pause events:
    modes = df.get_events('#state_system_mode')                                      # Find timestamps for run-time start and end (2=run)
    start_ev = [i for i in modes if i['value']==2][0]                                # 2=running, 0 or 1 = stopped

    run_idxs = [i for i,e in enumerate(modes) if e['time']>start_ev['time']]         # Get all "run states" if more than 1 found

    end_ev = next(i for i in modes[run_idxs[0]:] if i['value']==0 or i['value']==1)  # Find the first "stop" event after the first "run" event

    # Create a list of runs using start/stop-event times (so long as "Stop" button was not pressed during acquisition, only 1 chunk of time)
    bounds = []
    bounds.append([start_ev.time, end_ev.time])
    for r in run_idxs[1:]: 
        if modes[r].time < end_ev.time:  # Ignore any extra "run" events if there was no actual "stop" event
            continue
        else:                            # Otherwise, find the next "stop" event if any additional/new "run" events found.
            try:
                stop_ev = next(i for i in modes[r:] if i['value']==0 or i['value']==1)
            except StopIteration:
                end_event_name = 'trial_end'
                print "NO STOP DETECTED IN STATE MODES. Using alternative timestamp: %s." % end_event_name
                stop_ev = df.get_events(end_event_name)[-1]
                print stop_ev
            bounds.append([modes[r]['time'], stop_ev['time']])

    bounds[:] = [x for x in bounds if ((x[1]-x[0])/1E6)>1]
    # print "................................................................"
    print "****************************************************************"
    print "Parsing file\n%s... " % dfn
    print "Found %i start events in session." % len(bounds)
    print "Bounds: ", bounds
    for bidx, bound in enumerate(bounds):
        print "bound ID:", bidx, (bound[1]-bound[0])/1E6, "sec"
    print "****************************************************************"

    return df, bounds

In [6]:
def get_image_events(df, boundary, pixelclock_evs=[], stimtype='grating', mask=False):
    
    # Get all stimulus-udpate events within bounds:
    tmp_display_evs = df.get_events('#stimDisplayUpdate')                                                  # Get all stimulus-display-update events
    display_evs = [e for e in tmp_display_evs if e.value and not e.value[0]==None]                         # Filter out empty display-update events
    display_evs = [d for d in display_evs if d.time <= boundary[1] and d.time >= boundary[0]]              # Only include display-update events within time boundary of the session

    if len(pixelclock_evs)>0:
        pixelclock_evs = [i for i in display_evs for v in i.value if 'bit_code' in v.keys()]
        num_non_stimuli = 3 # N stimuli on screen: pixel clock, background, image
    if len(pixelclock_evs)==0:
        print "No pixel clock."
        pixelclock_evs = display_evs
        num_non_stimuli = 2 # N stimuli on screen: background, image
       
    # Get stimulus-onset info parsed into trials:
    if stimtype=='image':
        tmp_image_evs = [d for d in pixelclock_evs for i in d.value if 'filename' in i.keys() and '.png' in i['filename']]
        #stimevents.append(imdevs)

        # Find blank ITIs:
        if mask is True:
            iti_evs = [i for i in pixelclock_evs for v in i.value if v['name']=='blue_mask' and i not in tmp_image_evs]
        else:
            iti_evs = [i for i in pixelclock_evs if i.time>image_evs[0].time and i not in tmp_image_evs]

        tmp_trial_evs = tmp_image_evs + iti_evs
        trial_evs = sorted(tmp_trial_evs, key=get_timekey)
        
        image_evs = tmp_image_evs

    elif stimtype=='grating':
        #tmp_image_evs = [d for d in display_evs for i in d.value if i['name']=='gabor']
        tmp_image_evs = [d for d in display_evs for i in d.value if i['type']=='drifting_grating']
        
        start_times = [i.value[1]['start_time'] for i in tmp_image_evs] # Use start_time to ignore dynamic pixel-code of drifting grating since stim as actually static
        find_static = np.where(np.diff(start_times) > 0)[0]
        find_static = np.append(find_static, 0)
        find_static = sorted(find_static)
        image_evs = [tmp_image_evs[i+1] for i in find_static]
        print "got image events"
        #stimevents.append(imtrials)
        
        # Make sure only the 1st stimulus after a new-stim flag is counted (for guarantee-reward) experiments:
        newstim_evs = df.get_events('new_stimulus')
        new_stim_evs = [i for i in newstim_evs if i.value==1]
        print "N new-stimulus events:", len(new_stim_evs)
        first_image_idxs = []
        for idx,newev in enumerate(new_stim_evs[0:-1]):
            possible_image_evs = [i for i,ev in enumerate(image_evs) if ev.time>newev.time and ev.time<new_stim_evs[idx+1].time]
            first_image_idxs.append(possible_image_evs)
        first_image_idxs = [i[0] for i in first_image_idxs if len(i)>0]
        image_evs = [image_evs[i] for i,ev in enumerate(image_evs) if i in first_image_idxs]
        print "N image evs after taking only first image: ", len(image_evs)
        
        # Filter out image-events that were aborted:
        aborted_evs = df.get_events('trial_aborted')
        aborted_idxs = []
        aborted_evs = []
        for idx,imev in enumerate(image_evs[0:-1]):
            check_abort = [i.value for i in aborted_evs if i.time>imev.time and i.time<image_evs[idx+1].time]
            if sum(check_abort)>0:
                aborted_idxs.append(idx)
                aborted_evs.append(imev)
        print "N aborted images: ", len(aborted_idxs)
        image_evs = [image_evs[i] for i,ev in enumerate(image_evs) if i not in aborted_idxs]
        print "N image evs after removing aborted: ", len(image_evs)
        
        # Find blank ITIs:
        if mask is True:
            iti_evs = [i for i in pixelclock_evs if i.time>tmp_image_evs[0].time and i not in tmp_image_evs]
        else:
#             prevdev = [[i for i,d in enumerate(display_evs) if d.time < t.time][-1] for t in image_evs[1:]]
#             lastdev = [i for i,d in enumerate(display_evs) if d.time > image_evs[-1].time and len(d.value)<num_non_stimuli] # ignore the last "extra" ev (has diff time-stamp) - just wnt 1st non-grating blank
#             iti_evs = [display_evs[i] for i in prevdev]
#             if len(lastdev)>0:
#                 iti_evs.append(display_evs[lastdev[0]])
            #nonstim_evs = [i for i in display_evs if i not in tmp_image_evs]
            
            im_idx = [[t.time for t in display_evs].index(i.time) for i in image_evs]
            iti_evs = []
            for im in im_idx:
                try:
                    next_iti = next(i for i in display_evs[im:] if len(i.value)==(num_non_stimuli-1))
                    iti_evs.append(next_iti)
                except StopIteration:
                    print "No ITI found after this (should be last image_ev).\n"
                    #print display_evs[im]
            print "got iti events"
        
        # Check that we got all the blanks:
#         blanks = [i for i,p in enumerate(pixelclock_evs) if len(p.value)==(num_non_stimuli-1)]
#         mismatches = [i for i,(p,t) in enumerate(zip([pixelclock_evs[x] for x in blanks], iti_evs)) if not p==t]   
#         if len(mismatches)>0:
#             print "Mismatches found in parsing trials...."
#             print mismatches

        # Append a "trial off" at end, if needed:
        if image_evs[-1].time > iti_evs[-1].time: # early-abort
            print "Removing extra image event that has no offset."
            image_evs.pop(-1)                
        tmp_trial_evs = image_evs + iti_evs
        trial_evs = sorted(tmp_trial_evs, key=get_timekey)

    #trialevents.append(tmp_trialevents)
    print "Length of trial epochs: ", len(trial_evs)
    print "Number of trials found: ", len(image_evs)
    
    return image_evs, trial_evs, aborted_evs

In [7]:
def get_session_info(df):
    info = dict()
    
    stimon = df.get_events('stim_on_time')
    info['stimulus_duration'] = stimon[-1].value
    itis = df.get_events('ITI_duration')
    info['iti_duration'] = itis[-1].value
    sizes = df.get_events('fullscreen_size')
    info['stimsize'] = sizes[-1].value
    # stimulus types?
    # ntrials?
    portside = df.get_events('A_port')
    info['A_port'] = portside[-1].value
    
    return info

In [8]:
# From pymworks:
class Stimulus(object):
    def __init__(self, time, attributes, pixelclock=None):
        self.time = time
        self.attributes = attributes
        if pixelclock is not None:
            self.attributes['bit_code'] = pixelclock['bit_code']
        self.duration = None

    def __cmp__(self, other):
        return dict.__cmp__(self.attributes, other.attributes)

    def to_dict(self):
        d = self.attributes.copy()
        d.update(dict(time=self.time, duration=self.duration))
        return d

In [9]:
blacklisttests = [
    #lambda s: (('name' in s.keys()) and (s['name'] == 'pixel clock')),
    lambda s: (('type' in s.keys()) and (s['type'] == 'blankscreen')),
    ]

def to_stims(events, as_dicts=True, blacklist=None):
    if blacklist is None:
        blacklist = blacklisttests
    if not isinstance(blacklist, (tuple, list)):
        blacklist = (blacklist, )
    stims = []
    onscreen = []
    for e in sorted(events, key=lambda e: e.time):
        if e.value is None:
            logging.warning("Encountered event with value == None")
            if onscreen != {}:
                logging.error("Event.value == None with items on screen")
            continue
        current = []
        if hasattr(e.value, '__getitem__'):
            stimulus = None
            pixelclock = None
            for stim in e.value:
                if not isinstance(stim, dict) or \
                        any([t(stim) for t in blacklist]):
                    continue
                if ('name' in stim.keys()) and (stim['name'] == 'pixel clock'):
                    pixelclock = stim
                else:
                    if stimulus is not None:
                        logging.warning(
                            "Two stimuli onscreen: %s, %s"
                            % (stimulus, stim))
                    stimulus = stim
            if stimulus is not None:
                current.append(Stimulus(e.time, stimulus, pixelclock))
        #newstims, onscreen = find_stims(onscreen, current, e.time)
        stims += current #newstims
    if as_dicts:
        return [s.to_dict() for s in stims]
    return stims

In [10]:
def to_trials(stim_display_events, outcome_events, remove_unknown=True,
              duration_multiplier=2, stim_blacklists=None):
    """
    If remove_unknown, any trials where a corresponding outcome_event cannot
    be found will be removed.

    If duration_multiplier is not None, to_trials will check to see if the
    outcome event occured within duration_multiplier * duration microseconds
    of the trial start. If the outcome event occured later, the trial outcome
    will be marked as unknown.
    """
    if (len(outcome_events) == 0) or (len(stim_display_events) == 0):
        return []
    assert hasattr(outcome_events[0], 'name')

    trials = to_stims(stim_display_events, as_dicts=True,
                      blacklist=stim_blacklists)

    if (len(trials) == 0):
        return []

    outcomes = pymworks.events.utils.sync(outcome_events, trials,
                          direction=1, mkey=lambda x: x['time'])

    assert len(trials) == len(outcomes)
    unknowns = []
    if duration_multiplier is None:
        dtest = lambda t, o: True
    else:
        dtest = lambda t, o: \
            o.time < (t['time'] + t['duration'] * duration_multiplier)
    for i in xrange(len(trials)):
        if (outcomes[i] is not None) and dtest(trials[i], outcomes[i]):
            trials[i]['outcome'] = outcomes[i].name
            trials[i]['response_time'] = outcomes[i].time
        else:
            if remove_unknown:
                unknowns.append(i)
            else:
                trials[i]['outcome'] = 'unknown'

    # remove trials with 'unknown' outcome, in reverse
    for u in unknowns[::-1]:
        del trials[u]

    return trials

In [11]:
def get_first_response(found_outcome_evs):
    true_outcome_events = [f[0] for f in found_outcome_evs]
    first_outcome_evs = sorted([i for i in true_outcome_events], key=lambda e: e.time)

    success_evs = df.get_events('n_successes_session')
    fail_evs = df.get_events('n_failures_session')
    ignore_evs = df.get_events('n_ignores_session')

    result_events = []
    for idx,oev in enumerate(first_outcome_evs):
        if idx==len(first_outcome_evs)-1:  # don't need upper bound on time, since last outcome event:
            tmp_success_ev = [i for i in success_evs if i.time > oev.time]
            tmp_fail_ev = [i for i in fail_evs if i.time > oev.time]
            tmp_ignore_ev = [i for i in ignore_evs if i.time > oev.time]
        else:
            tmp_success_ev = [i for i in success_evs if i.time > oev.time and i.time < first_outcome_evs[idx+1].time]
            tmp_fail_ev = [i for i in fail_evs if i.time > oev.time and i.time < first_outcome_evs[idx+1].time]
            tmp_ignore_ev = [i for i in ignore_evs if i.time > oev.time and i.time < first_outcome_evs[idx+1].time]
        #print [len(tmp_success_ev), len(tmp_fail_ev), len(tmp_ignore_ev)]
        tmp_result_evs = [tmp_success_ev, tmp_fail_ev, tmp_ignore_ev]
        tmp_result_evs = [i[0] for i in tmp_result_evs if len(i)>0] # only take the first occurrence
        result_times = [(idx, i.time) for idx,i in enumerate(tmp_result_evs) if len(i)>0]  # get the time-stamps of first response in each type recorded
        result_idx = [ev for ev in result_times if ev[1]==min([i[1] for i in result_times])][0] # get the first response given
        result_ev = tmp_result_evs[result_idx[0]]
        result_events.append(result_ev)
    print "N first-outcomes found: ", len(first_outcome_evs)
    print "N outcomes (final): ", len(result_events)
    
    return result_events


In [12]:
# Set data-paths:
ismac = True
if ismac:
    prepend = '/Users/julianarhee'
else:
    prepend = '/'
source_dir = 'nas/volume1/behavior/data/mworks-data/head_fixed'
experiment = 'gratings'
version = 'v1'
nanimals = 4

data_dir = os.path.join(prepend, source_dir, experiment, version)
tmp_fns = os.listdir(data_dir)
fns = [f for f in tmp_fns if f.endswith('.mwk') and f.startswith('2017') and os.path.isfile(os.path.join(data_dir, f))]
print "Found %i MW data files." % len(fns)

version_output = '%s_output' % version
output_dir = os.path.join(prepend, source_dir, experiment, version_output)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

Found 36 MW data files.


In [13]:
session_dates = list(set([f[:-4].split('_')[0] for f in fns]))
print "Sessions: ", session_dates

animal_names = list(set([f[:-4].split('_')[1] for f in fns]))
if len(animal_names)>nanimals:
    print "More than %i expected IDs found:" % nanimals
    for a,name in enumerate(animal_names):
        print a, ": ", name
    discard = raw_input("Enter idx of repeat names:\n")
    if len(discard)>0:
        discard = int(discard)
        print "Removing ID: %s" % animal_names[discard]
        animal_names.pop(discard)

print "Animals: ", animal_names

Sessions:  ['20170810', '20170808', '20170809', '20170806', '20170807', '20170804', '20170805', '20170802', '20170803']
More than 4 expected IDs found:
0 :  Minerva
1 :  Hera
2 :  Athena
3 :  Juno
4 :  Juno2
Enter idx of repeat names:
4
Removing ID: Juno2
Animals:  ['Minerva', 'Hera', 'Athena', 'Juno']


In [14]:
# Create dict of data for all animals and sessions:
datafiles = dict()
for animal in animal_names:
    animal_files = [f for f in fns if animal in f]
    datafiles[animal] = dict((session, dict()) for session in session_dates)
    for session in datafiles[animal].keys():
        session_files = [os.path.join(data_dir, f) for f in animal_files if session in f]
        if len(session_files)==0:
            datafiles[animal].pop(session, None)
        else:
            datafiles[animal][session]['files'] = session_files
datafiles

{'Athena': {'20170802': {'files': ['/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170802_Athena.mwk']},
  '20170803': {'files': ['/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170803_Athena.mwk']},
  '20170804': {'files': ['/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170804_Athena.mwk']},
  '20170805': {'files': ['/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170805_Athena.mwk']},
  '20170806': {'files': ['/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170806_Athena.mwk']},
  '20170807': {'files': ['/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170807_Athena.mwk']},
  '20170808': {'files': ['/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170808_Athena.mwk']},
  '20170809': {'files': ['/Users/julianarhee/nas/volume1/behavior/data/mworks-da

In [15]:
# Check if data dict exists.  If not, create new.  If yes, load and append:
output_files = [i for i in os.listdir(output_dir) if version in i and i.endswith('parsed_trials.pkl')]
if len(output_files)>0:
    data_fn = output_files[0]
    print "Found previous output file. Loading: %s" % data_fn
    with open(os.path.join(output_dir, data_fn), 'rb') as f:
        data = pkl.load(f)
else:
    print "No previous output file found. Creating new."
    data = dict((animal, dict((session, dict()) for session in sorted(datafiles[animal].keys()))) for animal in animal_names)
    
print data.keys()

Found previous output file. Loading: v1_parsed_trials.pkl
['Minerva', 'Hera', 'Juno', 'Athena']


In [16]:
# Remove old datafiles, don't need to reprocess:
for animal in data.keys():
    previous_sessions = sorted(data[animal].keys())
    new_sessions = sorted([k for k in datafiles[animal].keys() if k not in previous_sessions])
    print "Found new sessions: "
    for s,sid in enumerate(new_sessions):
        print s, sid
        

Found new sessions: 
Found new sessions: 
Found new sessions: 
Found new sessions: 


In [17]:
# Look at specific animal datafile:
#animal = datafiles.keys()[2]
for animal in datafiles.keys():
    for session in sorted(new_sessions): #sorted(datafiles[animal].keys()):
        # session = sorted(datafiles[animal].keys())[2]

        print "Current session: ", session
        data[animal][session] = dict()

        files = datafiles[animal][session]['files']

        data[animal][session]['trials'] = []
        data[animal][session]['image_events'] = []
        data[animal][session]['result_events'] = []
        data[animal][session]['abort_events'] = []
        outcome_types = ['nsuccess', 'nfail', 'nignore', 'ntrials']
        data[animal][session]['performance'] = dict((outcome_type, 0) for outcome_type in outcome_types)
        data[animal][session]['performance']['unknown'] = [] 

        for fidx,f in enumerate(files):

            # ------------------------------------------------------------
            # Get session boundaries from file:
            # ------------------------------------------------------------
            df, bounds = get_session_bounds(f)
            t_boundary = bounds[0]

            if fidx==0:
                data[animal][session]['start_time'] = t_boundary[0]
                data[animal][session]['end_time'] = t_boundary[1]
            else:
                if t_boundary[0] < data[animal][session]['start_time']:
                    # 2nd processed file actually occurred earlier in time
                    data[animal][session]['start_time'] = t_boundary[0]
                    append_before = True
                else:
                    data[animal][session]['end_time'] = t_boundary[1]
                    append_before = False

            # ------------------------------------------------------------
            # Get session info:
            # ------------------------------------------------------------
            session_info = get_session_info(df)
            print session_info
            if not 'info' in data[animal][session].keys():
                data[animal][session]['info'] = session_info

            # ------------------------------------------------------------
            # Get training info:
            # ------------------------------------------------------------
            pre_reward_evs = df.get_events('flag_pre_reward')
            pre_reward_evs = [i for i in pre_reward_evs if i.time>= t_boundary[0] and i.time<=t_boundary[1]]
            # print prereward_flag
            guarantee_reward_evs = df.get_events('flag_guarantee_reward')
            guarantee_reward_evs = [i for i in guarantee_reward_evs if i.time>=t_boundary[0] and i.time<=t_boundary[1]]
            # print guarantee_flag
            if sum([ev.value for ev in pre_reward_evs])>0:
                check_prereward = True
            else:
                check_prereward = False
            if sum([ev.value for ev in guarantee_reward_evs])>0:
                check_guarantee = True
            else:
                check_guarantee = False

            # ------------------------------------------------------------
            # Get all trials (and stimulu-ITI events):
            # ------------------------------------------------------------
            image_events, all_trial_events, abort_events = get_image_events(df, t_boundary, stimtype='grating')

            # Get ITI events and check them:
            iti_events = [ev for ev in all_trial_events[1::2]]
            df.get_events()

            incorrect_iti = [idx for idx,i in enumerate(iti_events) if len(i.value)>1]
            if len(incorrect_iti)>0:
                print "Found %i incorrect blank-screen ITIs..." % len(incorrect_iti)
                print incorrect_iti

            # If no "blank" period after image, experiment ended mid-stimulus, so remove:
            if len(image_events)*2 > len(all_trial_events):
                image_events.pop(-1)
            print "N trials: ", len(image_events)
            print "N trial epochs: ",  len(all_trial_events)

            # Filter out any non-outcome trials:
            iti_events = [ev for ev in all_trial_events[1::2]]
            print "n imagess: ", len(image_events)
            print "n itis: ", len(iti_events)


            # ------------------------------------------------------------
            # Get all RESPONSE events:
            # ------------------------------------------------------------
            outcome_reached_evs = df.get_events('outcome_reached')
            found_outcome_evs = []
            for idx,(im,iti) in enumerate(zip(image_events, iti_events)):
                next_outcome = [i for i in outcome_reached_evs if i.time>im.time and i.time<iti.time and i.value==1]
                found_outcome_evs.append(next_outcome)

            print "N outcomes found: ", len(found_outcome_evs)

            # Before 08/03, all animals except (Athena?) ran on protocol that did not properly reset "trial_aborted" flag
            # This means that some new-stimulus events could be "aborted" trials that are not caught by the aborted-trial flag.
            # If no outcome is reached between new-stimulus events in image_events (and all_trial_events), additionally remove those:
            print "N new-stimulus trial starts: ", len(image_events)
            if len(found_outcome_evs)!=len(image_events):
                print "Mismatch in N outcomes and N image-events found."

            # Check for extra aborts:
            extra_aborts = [idx for idx,ev in enumerate(found_outcome_evs) if len(ev)==0]
            if len(extra_aborts)>0:
                print "Found %i additional aborted trials (no outcome reached after stimulus shown)." % len(extra_aborts)
                print extra_aborts

            if session=='20170802' or len(extra_aborts)>0:
                print "Removing additional trial-abort events."
                if len(found_outcome_evs)==len(image_events):
                    # Remove extra aborts from result-evs and image-evs:
                    found_outcome_evs = [ev for idx,ev in enumerate(found_outcome_evs) if idx not in extra_aborts]
                    image_events = [ev for idx,ev in enumerate(image_events) if idx not in extra_aborts]
                    extra_aborted_events = [ev for idx,ev in enumerate(image_events) if idx in extra_aborts]
                    abort_events.append(extra_aborted_events)
                    abort_events = [item for sublist in abort_events for item in sublist]

                    # Also remove from all-trial-epochs list:
                    extra_aborts_all_epochs = [[i*2, i*2+1] for i in extra_aborts]
                    extra_aborts_all_epochs = [item for sublist in extra_aborts_all_epochs for item in sublist]
                    all_trial_events = [ev for idx,ev in enumerate(all_trial_events) if idx not in extra_aborts_all_epochs]

            #print len(image_events)
            #print len(found_outcome_evs)

            # ------------------------------------------------------------
            # Get animal's FIRST response:
            # ------------------------------------------------------------
            # Since training flags influence which and how many outcomes are possible on each trial, need to distinguish:
            # first response from all responses. For example, an initial "ignore" or "fail" trial can still have a "success"
            # if training flag flag_guarantee_reward is on.
            result_events = get_first_response(found_outcome_evs)

            # ------------------------------------------------------------
            # Assign FIRST outcome to each trial:
            # ------------------------------------------------------------
            # tmp_trial_events = pymworks.events.display.to_trials(image_events, result_evs, duration_multiplier=1, remove_unknown=False)
            triallist = to_trials(image_events, result_events, duration_multiplier=None,remove_unknown=False)
                
            print "N result events: ", len(result_events)
            print "N trial events: ", len(triallist)
            mismatch = []
            for idx,(res,tri) in enumerate(zip(result_events, triallist[0:len(result_events)])):
                if not res.name==tri['outcome']:
                    mismatch.append(idx)
            if len(mismatch)>0:
                print "-----------------------WARNING_--------------------------"
                print "---------------------------------------------------------"
                print "Bad stimulus-to-response alignment. Check pymworks functions or -duration- arugment to to_trials()."
                print "Found %i mismatches, starting at index %i." % (len(mismatch), mismatch[0])
                print "---------------------------------------------------------"
            
            # ------------------------------------------------------------
            # Check training flags, if needed:
            # ------------------------------------------------------------
            if check_prereward:
                prereward_on = [i.time for i in pre_reward_evs if i.value==1]
                prereward_off = [i.time for i in pre_reward_evs if i.value==0 and i.time>min(prereward_on)]
                if len(prereward_off)==0 and len(prereward_on)>0:
                    prereward_off.append(t_boundary[1])
                print "pre-reward ON (s): ", (max(prereward_off)-min(prereward_on))/1E6
                for t in triallist:
                    if t['time']<=max(prereward_off) and i['time']>=min(prereward_on):
                        t['prereward'] = 1
                    else:
                        t['prereward'] = 0
            else:
                for t in triallist:
                    t['prereward'] = 0
            if check_guarantee:
                guarantee_on = [i.time for i in guarantee_reward_evs if i.value==1]
                guarantee_off = [i.time for i in guarantee_reward_evs if i.value==0]
                #if min(guarantee_on)
                if len(guarantee_off)==0 and len(guarantee_on)>0:
                    guarantee_off.append(t_boundary[1])
                print "guarantee ON (s): ", (max(guarantee_off)-min(guarantee_on))/1E6
                for t in triallist:
                    if t['time']<=max(guarantee_off) and i['time']>=min(guarantee_on):
                        t['guarantee'] = 1
                    else:
                        t['guarantee'] = 0
            else:
                for t in triallist:
                    t['guarantee'] = 0
            print triallist[0]
            
            # ------------------------------------------------------------
            # Store event info:
            # ------------------------------------------------------------
            #if 'trials' in data[animal][session].keys():
            data[animal][session]['trials'].append(triallist)
            data[animal][session]['image_events'].append(image_events)
            data[animal][session]['result_events'].append(result_events)
            data[animal][session]['abort_events'].append(abort_events)

            # ------------------------------------------------------------
            # Tally up each outcome type:
            # ------------------------------------------------------------
            unknown = []
            nsuccess = 0; nfail = 0; nignore = 0; ntrials = 0
            for idx,t in enumerate(triallist):
                ntrials += 1
                if 'success' in t['outcome']:
                    nsuccess += 1
                elif 'fail' in t['outcome']:
                    nfail += 1
                elif 'ignore' in t['outcome']:
                    nignore += 1
                else:
                    unknown.append(idx)
                    

            data[animal][session]['performance']['ntrials'] += ntrials
            data[animal][session]['performance']['nsuccess'] += nsuccess
            data[animal][session]['performance']['nfail'] += nfail
            data[animal][session]['performance']['nignore'] += nignore
            data[animal][session]['performance']['unknown'] = data[animal][session]['performance']['unknown'] + unknown

        # Flatten and sort lists by time:
        if f>1:
            if len(triallist)>0:
                data[animal][session]['trials'] = sorted([item for sublist in data[animal][session]['trials'] for item in sublist], key=lambda e: e['time'])
            if len(image_events)>0:
                data[animal][session]['image_events'] = sorted([item for sublist in data[animal][session]['image_events'] for item in sublist], key=lambda e: e.time)
            if len(result_events)>0:
                data[animal][session]['result_events']= sorted([item for sublist in data[animal][session]['result_events'] for item in sublist], key=lambda e: e.time)
            if len(abort_events)>0:
                data[animal][session]['abort_events'] = sorted([item for sublist in data[animal][session]['abort_events'] for item in sublist], key=lambda e: e.time)
        #     print "N successes: ", nsuccess
        #     print "N fails: ", nfail
        #     print "N ignores: ", nignore
        #     print "N trials: ", ntrials


In [18]:
# Save processed data:
output_fn = '%s_parsed_trials.pkl' % version
with open(os.path.join(output_dir, output_fn), 'wb') as f:
    pkl.dump(data, f)

print "Saved output to: ", os.path.join(output_dir, output_fn)

Saved output to:  /Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1_output/v1_parsed_trials.pkl


In [19]:
# Plot overall performance:
from bokeh.io import gridplot

c_success = "navy"
c_fail = "firebrick"
c_ignore = "gray"
lwidth = 2
flagpos1 = 1.1; flagcol1 = "black";
flagpos2 = 1; flagcol2 = "black";
flagsize = 5

#animal = data.keys()[0]
# create a new plot
subplots = []
for animal in data.keys():
    ntrials = [data[animal][s]['performance']['ntrials'] for s in sorted(data[animal].keys())]
    ncorrect = [data[animal][s]['performance']['nsuccess'] for s in sorted(data[animal].keys())]
    nfail = [data[animal][s]['performance']['nfail'] for s in sorted(data[animal].keys())]
    nignore = [data[animal][s]['performance']['nignore'] for s in sorted(data[animal].keys())]
    nsessions = len(sorted(data[animal].keys()))
    s1 = figure(width=250, plot_height=250, title=animal)
    
    nattempted = np.array([float(n) for n in nfail]) + np.array([float(n) for n in ncorrect])
    ncorrect = np.array([float(n) for n in ncorrect])/ nattempted #np.array([float(t) for t in ntrials])
    #nfail = np.array([float(n) for n in nfail])/np.array([float(t) for t in ntrials])
    nignore = np.array([float(n) for n in nignore])/np.array([float(t) for t in ntrials])
    
    s1.line(range(nsessions), ncorrect, line_width=lwidth, line_color=c_success, line_alpha=0.5)
    #s1.line(range(nsessions), nfail, line_width=lwidth, line_color=c_fail, line_alpha=0.5)
    s1.line(range(nsessions), nignore, line_width=lwidth, line_color=c_ignore, line_alpha=0.5)

    n_prereward_on = [sum(data[animal][s]['trials'][t]['prereward'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
    n_guarantee_on = [sum(data[animal][s]['trials'][t]['guarantee'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
    percent_prereward_on = np.array([float(n) for n in n_prereward_on])/np.array([float(n) for n in ntrials])
    percent_guarantee_on = np.array([float(n) for n in n_guarantee_on])/np.array([float(n) for n in ntrials])
    flag_prereward = [idx for idx,p in enumerate(percent_prereward_on) if p>0.25]
    flag_guarantee = [idx for idx,p in enumerate(percent_guarantee_on) if p>0.25]
    s1.circle(flag_prereward, np.ones((len(flag_prereward),))*flagpos1, size=flagsize, color=flagcol1, alpha=0.5)
    s1.triangle(flag_guarantee, np.ones((len(flag_guarantee),))*flagpos2, size=flagsize, color=flagcol2, alpha=0.5)
    
    
    subplots.append(s1)


p = gridplot([[subplots[0], subplots[1]], [subplots[2], subplots[3]]])
show(p)

#print session_dates
print sorted(data['Athena'].keys())
# # Create another plot:
# animal = data.keys()[1]
# # create a new plot
# ntrials = [data[animal][s]['performance']['ntrials'] for s in sorted(data[animal].keys())]
# ncorrect = [data[animal][s]['performance']['nsuccess'] for s in sorted(data[animal].keys())]
# nfail = [data[animal][s]['performance']['nfail'] for s in sorted(data[animal].keys())]
# nignore = [data[animal][s]['performance']['nignore'] for s in sorted(data[animal].keys())]
# nsessions = len(sorted(data[animal].keys()))
# n_prereward_on = [sum(data[animal][s]['trials'][t]['prereward'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
# n_guarantee_on = [sum(data[animal][s]['trials'][t]['guarantee'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
# percent_prereward_on = np.array([float(n) for n in n_prereward_on])/np.array([float(n) for n in ntrials])
# percent_guarantee_on = np.array([float(n) for n in n_guarantee_on])/np.array([float(n) for n in ntrials])
# flag_prereward = [idx for idx,p in enumerate(percent_prereward_on) if p>0.5]
# flag_guarantee = [idx for idx,p in enumerate(percent_guarantee_on) if p>0.5]

# s2 = figure(width=250, plot_height=250, title=animal)
# ncorrect = np.array([float(n) for n in ncorrect])/np.array([float(t) for t in ntrials])
# nfail = np.array([float(n) for n in nfail])/np.array([float(t) for t in ntrials])
# nignore = np.array([float(n) for n in nignore])/np.array([float(t) for t in ntrials])
# s2.line(range(nsessions), ncorrect, line_width=lwidth, line_color=c_success, line_alpha=0.5)
# s2.line(range(nsessions), nfail, line_width=lwidth, line_color=c_fail, line_alpha=0.5)
# s2.line(range(nsessions), nignore, line_width=lwidth, line_color=c_ignore, line_alpha=0.5)
# s2.circle(flag_prereward, np.ones((len(flag_prereward),))*flagpos1, size=flagsize, color=flagcol1, alpha=0.5)
# s2.triangle(flag_guarantee, np.ones((len(flag_guarantee),))*flagpos2, size=flagsize, color=flagcol2, alpha=0.5)


# # Create another plot:
# animal = data.keys()[2]
# # create a new plot
# ntrials = [data[animal][s]['performance']['ntrials'] for s in sorted(data[animal].keys())]
# ncorrect = [data[animal][s]['performance']['nsuccess'] for s in sorted(data[animal].keys())]
# nfail = [data[animal][s]['performance']['nfail'] for s in sorted(data[animal].keys())]
# nignore = [data[animal][s]['performance']['nignore'] for s in sorted(data[animal].keys())]
# nsessions = len(sorted(data[animal].keys()))
# n_prereward_on = [sum(data[animal][s]['trials'][t]['prereward'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
# n_guarantee_on = [sum(data[animal][s]['trials'][t]['guarantee'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
# percent_prereward_on = np.array([float(n) for n in n_prereward_on])/np.array([float(n) for n in ntrials])
# percent_guarantee_on = np.array([float(n) for n in n_guarantee_on])/np.array([float(n) for n in ntrials])
# flag_prereward = [idx for idx,p in enumerate(percent_prereward_on) if p>0.5]
# flag_guarantee = [idx for idx,p in enumerate(percent_guarantee_on) if p>0.5]

# s3 = figure(width=250, plot_height=250, title=animal)
# ncorrect = np.array([float(n) for n in ncorrect])/np.array([float(t) for t in ntrials])
# nfail = np.array([float(n) for n in nfail])/np.array([float(t) for t in ntrials])
# nignore = np.array([float(n) for n in nignore])/np.array([float(t) for t in ntrials])

# s3.line(range(nsessions), ncorrect, line_width=lwidth, line_color=c_success, line_alpha=0.5)
# s3.line(range(nsessions), nfail, line_width=lwidth, line_color=c_fail, line_alpha=0.5)
# s3.line(range(nsessions), nignore, line_width=lwidth, line_color=c_ignore, line_alpha=0.5)
# s3.circle(flag_prereward, np.ones((len(flag_prereward),))*flagpos1, size=flagsize, color=flagcol1, alpha=0.5)
# s3.triangle(flag_guarantee, np.ones((len(flag_guarantee),))*flagpos2, size=flagsize, color=flagcol2, alpha=0.5)


# # Create another plot:
# animal = data.keys()[3]
# # create a new plot
# ntrials = [data[animal][s]['performance']['ntrials'] for s in sorted(data[animal].keys())]
# ncorrect = [data[animal][s]['performance']['nsuccess'] for s in sorted(data[animal].keys())]
# nfail = [data[animal][s]['performance']['nfail'] for s in sorted(data[animal].keys())]
# nignore = [data[animal][s]['performance']['nignore'] for s in sorted(data[animal].keys())]
# nsessions = len(sorted(data[animal].keys()))

# n_prereward_on = [sum(data[animal][s]['trials'][t]['prereward'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
# n_guarantee_on = [sum(data[animal][s]['trials'][t]['guarantee'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
# percent_prereward_on = np.array([float(n) for n in n_prereward_on])/np.array([float(n) for n in ntrials])
# percent_guarantee_on = np.array([float(n) for n in n_guarantee_on])/np.array([float(n) for n in ntrials])
# flag_prereward = [idx for idx,p in enumerate(percent_prereward_on) if p>0.5]
# flag_guarantee = [idx for idx,p in enumerate(percent_guarantee_on) if p>0.5]

# s4 = figure(width=250, plot_height=250, title=animal)
# nattempted = np.array(ncorrect) + np.array(nfail)
# ncorrect = np.array([float(n) for n in ncorrect])/ nattempted #np.array([float(t) for t in ntrials])
# #nfail = np.array([float(n) for n in nfail])/np.array([float(t) for t in ntrials])
# nignore = np.array([float(n) for n in nignore])/np.array([float(t) for t in ntrials])

# s4.line(range(nsessions), ncorrect, line_width=lwidth, line_color=c_success, line_alpha=0.5)
# s4.line(range(nsessions), nfail, line_width=lwidth, line_color=c_fail, line_alpha=0.5)
# s4.line(range(nsessions), nignore, line_width=lwidth, line_color=c_ignore, line_alpha=0.5)
# s4.circle(flag_prereward, np.ones((len(flag_prereward),))*flagpos1, size=flagsize, color=flagcol1, alpha=0.5)
# s4.triangle(flag_guarantee, np.ones((len(flag_guarantee),))*flagpos2, size=flagsize, color=flagcol2, alpha=0.5)


# p = gridplot([[s1,s2], [s3,s4]])
# show(p)

['20170802', '20170803', '20170804', '20170805', '20170806', '20170807', '20170808', '20170809', '20170810']


In [20]:
data[animal]['20170810']['performance']
s='20170810'

n_prereward_on = [sum(data[animal][s]['trials'][t]['prereward'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
n_guarantee_on = [sum(data[animal][s]['trials'][t]['guarantee'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]

print len(n_prereward_on)

percent_prereward_on = np.array([float(n) for n in n_prereward_on])/np.array([float(n) for n in ntrials])
percent_guarantee_on = np.array([float(n) for n in n_guarantee_on])/np.array([float(n) for n in ntrials])
flag_prereward = [idx for idx,p in enumerate(percent_prereward_on) if p>0.25]
flag_guarantee = [idx for idx,p in enumerate(percent_guarantee_on) if p>0.25]
print percent_prereward_on


nattempted = np.array(ncorrect) + np.array(nfail)
perc_correct = np.array(ncorrect)/nattempted #np.array(ntrials)
#perc_fail = np.array(nfail)/np.array(ntrials)
perc_ignore = np.array(nignore)/np.array(ntrials)

print perc_ignore



9
[ 1.          0.09731544  0.12820513  0.01754386  0.          0.          0.
  0.          0.        ]
[ 0.00016194  0.0010191   0.0016117   0.00075985  0.00023529  0.00021403
  0.00036505  0.00024526  0.000128  ]


In [21]:
# Create average plot of performance:
response_types = ['correct', 'ignore'] # 'fail', 'ignore']
all_means = dict((resp, dict()) for resp in response_types)
included_sessions= dict()
# percent-Hit & num-Hit-Trials?
for animal in data.keys():

    ntrials = [float(data[animal][s]['performance']['ntrials']) for s in sorted(data[animal].keys())]
    ncorrect = [float(data[animal][s]['performance']['nsuccess']) for s in sorted(data[animal].keys())]
    nfail = [float(data[animal][s]['performance']['nfail']) for s in sorted(data[animal].keys())]
    nignore = [float(data[animal][s]['performance']['nignore']) for s in sorted(data[animal].keys())]
    
    n_prereward_on = [sum(data[animal][s]['trials'][t]['prereward'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
    n_guarantee_on = [sum(data[animal][s]['trials'][t]['guarantee'] for t in range(len(data[animal][s]['trials']))) for s in sorted(data[animal].keys())]
    percent_prereward_on = np.array([float(n) for n in n_prereward_on])/np.array([float(n) for n in ntrials])
    percent_guarantee_on = np.array([float(n) for n in n_guarantee_on])/np.array([float(n) for n in ntrials])
    flag_prereward = [idx for idx,p in enumerate(percent_prereward_on) if p>0.25]
    flag_guarantee = [idx for idx,p in enumerate(percent_guarantee_on) if p>0.25]
    
#     filtered_ncorrect = [v for idx,v in enumerate(ncorrect) if idx not in flag_prereward]
#     filtered_nfail = [v for idx,v in enumerate(nfail) if idx not in flag_prereward]
#     filtered_nignore = [v for idx,v in enumerate(nignore) if idx not in flag_prereward]
#     filtered_ntrials = [v for idx,v in enumerate(ntrials) if idx not in flag_prereward]
    
#     nattempted = np.array(filtered_ncorrect) + np.array(filtered_nfail)
#     perc_correct = np.array(filtered_ncorrect)/nattempted #np.array(ntrials)
#     perc_fail = np.array(filtered_nfail)/np.array(filtered_ntrials)
#     perc_ignore = np.array(filtered_nignore)/np.array(filtered_ntrials)
    
    nattempted = np.array(ncorrect) + np.array(nfail)
    perc_correct = np.array(ncorrect)/nattempted #np.array(ntrials)
    #perc_fail = np.array(nfail)/np.array(ntrials)
    perc_ignore = np.array(nignore)/np.array(ntrials)
    
    all_means['correct'][animal] = [noutcome for flag,noutcome in zip(percent_prereward_on, perc_correct) if flag<0.25]
    #all_means['fail'][animal] = [noutcome for flag,noutcome in zip(percent_prereward_on, perc_fail) if flag<0.5]
    all_means['ignore'][animal] = [noutcome for flag,noutcome in zip(percent_prereward_on, perc_ignore) if flag<0.25]
    #all_means['sessions'][animal] = [idx for idx,(flag,noutcome) in enumerate(zip(percent_prereward_on, perc_ignore)) if flag<0.25]
    included_sessions[animal] = [idx for idx,(flag,noutcome) in enumerate(zip(percent_prereward_on, perc_ignore)) if flag<0.25]

In [22]:
included_sessions

{'Athena': [1, 2, 3, 4, 5, 6, 7, 8],
 'Hera': [1, 2, 3, 4, 5, 6, 7, 8],
 'Juno': [0, 2, 3, 4, 5, 6, 7, 8],
 'Minerva': [0, 1, 2, 3, 4, 5, 6, 7]}

In [23]:
print response_types
session_dates = sorted(data['Athena'].keys())

max_sessions = max([len(included_sessions[i]) for i in included_sessions.keys()])
summary = dict((resp, dict()) for resp in response_types)
for resp in summary.keys():
    session_means = dict((session,[]) for session in range(max([len(included_sessions[i]) for i in included_sessions.keys()])))#sorted(session_dates))
    session_errs = dict((session,[]) for session in range(max([len(included_sessions[i]) for i in included_sessions.keys()])))#session_dates)
    for sidx in range(max_sessions):
        session_mean_correct = [all_means[resp][animal][sidx] for animal in data.keys() if len(all_means[resp][animal])>sidx]
        session_means[sidx] = np.mean(session_mean_correct)
        session_errs[sidx] = np.std(session_mean_correct)
        
#     for sidx,session in enumerate(sorted(session_dates)):
    
#         for animal in included_sessions.keys():
#             if sidx in included_session[animal]:
                
#         session_mean_correct = [all_means[resp][animal][sidx] for animal in data.keys() if len(all_means[resp][animal])>sidx]
#         print session_mean_correct
#         session_means[sidx] = np.mean(session_mean_correct)
#         session_errs[sidx] = np.std(session_mean_correct)
        #print session, session_means[session] #np.mean(session_mean_correct)
    summary[resp]['means'] = session_means
    summary[resp]['errs'] = session_errs

#print all_means

['correct', 'ignore']


In [24]:
print session_means
print session_errs

{0: 0.69815274323471055, 1: 0.54104502983416303, 2: 0.46354195058222403, 3: 0.48098708729147055, 4: 0.49192077987375449, 5: 0.48812505833049391, 6: 0.53084479899381354, 7: 0.50785835534003043}
{0: 0.094720177026782051, 1: 0.1110934168458261, 2: 0.048634308002411558, 3: 0.0085623266370658587, 4: 0.034064727078013619, 5: 0.047420509738451161, 6: 0.030316891073568129, 7: 0.035508098656313776}


In [25]:
print animal
summary[resp]

Athena


{'errs': {0: 0.094720177026782051,
  1: 0.1110934168458261,
  2: 0.048634308002411558,
  3: 0.0085623266370658587,
  4: 0.034064727078013619,
  5: 0.047420509738451161,
  6: 0.030316891073568129,
  7: 0.035508098656313776},
 'means': {0: 0.69815274323471055,
  1: 0.54104502983416303,
  2: 0.46354195058222403,
  3: 0.48098708729147055,
  4: 0.49192077987375449,
  5: 0.48812505833049391,
  6: 0.53084479899381354,
  7: 0.50785835534003043}}

In [26]:
# plot the points
response_colors = ["navy", "gray"] #"firebrick", "gray"]
resp_colors = dict((resp, c_resp) for resp,c_resp in zip(response_types, response_colors))

print resp_colors
p = figure(title='overall performance (n=4)', width=800, height=400, y_range=[0,1])
p.xaxis.axis_label = 'session'
p.yaxis.axis_label = 'percent'

for ridx,resp in enumerate(summary.keys()):
    session_means = summary[resp]['means']
    xs = range(len(session_means.keys()))
    print [s for s in sorted(session_means.keys())]
    ys = [session_means[s] for s in sorted(session_means.keys())]
    yerr = [session_errs[s] for s in sorted(session_errs.keys())]
    
    p.line(xs, ys, color=resp_colors[resp], line_width=2, line_alpha=0.5, legend=resp)
    #p.hold()
    #p.circle(xs, ys, color=resp_colors[resp], size=5, line_alpha=0)

    # create the coordinates for the errorbars
    err_xs = []
    err_ys = []

    for x, y, yerr in zip(xs, ys, yerr):
        err_xs.append((x, x))
        err_ys.append((y - yerr, y + yerr))

    # plot them
    p.multi_line(err_xs, err_ys, color=resp_colors[resp], line_width=2, line_alpha=0.5)
    
#     tmpx = [range(len(xs)), np.flipud(range(len(xs)))]
#     tmpx = [item for sublist in tmpx for item in sublist]
#     tmperr = [[e[0] for e in err_ys], np.flipud([e[1] for e in err_ys])]
#     tmperr = [item for sublist in tmperr for item in sublist]
#     p.patch(tmpx, tmperr, alpha=0.2, line_width=0, color=resp_colors[resp], legend=resp)

    
#p.legend.label_text_font = "times"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.major_tick_out = 5
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

    
show(p)
from bokeh.io import export_svgs

p.output_backend = "svg"
# export_svgs(p, filename="plot.svg")

{'ignore': 'gray', 'correct': 'navy'}
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 2, 3, 4, 5, 6, 7]


In [27]:
p.output_backend = "svg"
fnsvg = os.path.join(output_dir, "overall_performance_split.svg")
# export_svgs(p, filename="plot.svg")
export_svgs(p, filename=fnsvg)

['/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1_output/overall_performance_split.svg']

In [449]:
tmperr = [[e[0] for e in err_ys], np.flipud([e[1] for e in err_ys])]
tmperr = [item for sublist in tmperr for item in sublist]
print tmperr

[0.056811210306877587, 0.021444160959226868, 0.11397812652232424, 0.44711045988017695, 0.40290136520907815, 0.4035133440139142, 0.37728945399625285, 0.3808428759151492, 0.44571030232547493, 0.48326003817038715, 0.52288009059163876, 0.51067524815627696, 0.50706476570680725, 0.30119560673354373, 0.23588714946294731, 0.10033501507025544]


In [35]:
# Look at ITI performance

animal = datafiles.keys()[0]
# animal = 'Hera'
for animal in datafiles.keys():
     for session in sorted(datafiles[animal].keys()):
#         animal = 'Hera'
#         session= '20170805'
        files = datafiles[animal][session]['files']
        
        data[animal][session]['iti_events'] = []
        data[animal][session]['iti_start_events'] = []
        for fidx,f in enumerate(files):
           # print f
            df = None
            df, bounds = get_session_bounds(f)
            t_boundary = bounds[0]

            # IIT value increments for each time ITI-wait loop is entered. Last updated value is time of true ITI:
            iti_attempts = df.get_events('n_iti_attempts')
            iti_attempts = [ev for ev in iti_attempts if ev.time<=t_boundary[1] and ev.time>=t_boundary[0]]
            print len(iti_attempts)
            iti_restarts = [idx for idx,ev in enumerate(iti_attempts) if ev.value==0 and iti_attempts[idx+1].value==1]
            #print iti_restarts[0:10]
            iti_last_attempt = [i-1 for i in iti_restarts[1:]]
            iti_last_attempt.append(0)
            #print iti_last_attempt[0:5]
            tmp_iti_events = [ev for ix,ev in enumerate(iti_attempts) if ix in iti_last_attempt]
            #print len(tmp_iti_events)
            #print len(data[animal][session]['trials'])

            iti_dur = data[animal][session]['info']['iti_duration']
            iti_events = []
            trials = data[animal][session]['trials']
            subset_trials = [t for t in trials if t['time']>=t_boundary[0] and t['time']<=t_boundary[1]]
            for idx,trial in enumerate(sorted(subset_trials, key=lambda e: e['start_time'])):
                if idx==0:
                    tmp_iti_idx = [tidx for tidx,ev in enumerate(sorted(tmp_iti_events, key=lambda e: e.time)) if ev.time<=trial['start_time']+iti_dur]
                else:
                    tmp_iti_idx = [tidx for tidx,ev in enumerate(sorted(tmp_iti_events, key=lambda e: e.time)) if ev.time<=trial['start_time']+iti_dur and ev.time>subset_trials[idx-1]['time']]
                iti_match = tmp_iti_events[max(tmp_iti_idx)]
                iti_events.append(iti_match)
            print len(subset_trials)
            print len(iti_events)

            # also find ITI starts:
            iti_start_events = []
            for idx,iti_ev in enumerate(iti_events):
                if idx==0:
                    start_iti = [ev for ev in iti_attempts if ev.time<=iti_ev.time and ev.value==1]
                    if len(start_iti)==0: # bad start/stop
                        print len(iti_attempts[0])
                        start_iti = iti_attempts[0]
                else:
                    start_iti = [ev for ev in iti_attempts if ev.time<=iti_ev.time and ev.time>iti_events[idx-1].time and ev.value==1][0]
                iti_start_events.append(start_iti)
           
            print "N ITI starts: ", len(iti_start_events)
            print "N ITI events:", len(iti_events)

            data[animal][session]['iti_events'].append(iti_events)
            data[animal][session]['iti_start_events'].append(iti_start_events)

        if fidx>1:
            if len(iti_events)>0:
                data[animal][session]['iti_events'] = sorted([item for sublist in data[animal][session]['iti_events'] for item in sublist], key=lambda e: e.time)
            if len(iti_start_events)>0:
                data[animal][session]['iti_start_events'] = sorted([item for sublist in data[animal][session]['iti_start_events'] for item in sublist], key=lambda e: e.time)


****************************************************************
Parsing file
/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170802_Minerva.mwk... 
Found 1 start events in session.
Bounds:  [[3180770839, 7053242064]]
bound ID: 0 3872.471225 sec
****************************************************************
1535
516
516
N ITI starts:  516
N ITI events: 516
****************************************************************
Parsing file
/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170803_Minerva.mwk... 
Found 1 start events in session.
Bounds:  [[87393573863, 91054812488]]
bound ID: 0 3661.238625 sec
****************************************************************
1871
359
359
N ITI starts:  359
N ITI events: 359
****************************************************************
Parsing file
/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170805_Minerva.mwk... 
Found 1 start events in s

N ITI starts:  520
N ITI events: 520
****************************************************************
Parsing file
/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170805_Juno.mwk... 
Found 1 start events in session.
Bounds:  [[5728538426, 10012371756]]
bound ID: 0 4283.83333 sec
****************************************************************
1772
410
410
N ITI starts:  410
N ITI events: 410
****************************************************************
Parsing file
/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170806_Juno.mwk... 
Found 1 start events in session.
Bounds:  [[90283052707, 94667348638]]
bound ID: 0 4384.295931 sec
****************************************************************
1978
379
379
N ITI starts:  379
N ITI events: 379
****************************************************************
Parsing file
/Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1/20170807_Juno.mwk..

IndexError: list index out of range

In [353]:
len(data[animal][session]['iti_start_events'])
#len(data[animal][session]['trials'])
#[(iend.time - istart.time) for istart,iend in zip(iti_start_events, iti_events)]

0

In [354]:
# Save processed data:
output_fn = '%s_parsed_trials.pkl' % version
with open(os.path.join(output_dir, output_fn), 'wb') as f:
    pkl.dump(data, f)

print "Saved output to: ", os.path.join(output_dir, output_fn)


Saved output to:  /Users/julianarhee/nas/volume1/behavior/data/mworks-data/head_fixed/gratings/v1_output/v1_parsed_trials.pkl


In [357]:
# Plot histogram of ITI durations > min:
session_dates = data['Athena'].keys();

iti_by_session = dict()
for animal in data.keys():
    session_itis = []
    for session in data[animal].keys(): #sorted(session_dates):
        #print animal, session
        if session in data[animal].keys():
            iti_start_events = data[animal][session]['iti_start_events']
            if len(iti_start_events[0])==1:
                iti_start_events[0] = iti_start_events[0][0]
            iti_events = data[animal][session]['iti_events'][0]
            #print len(iti_start_events)
            #print len(iti_events)
            iti_overtime = [(iend.time - istart.time) for istart,iend in zip(iti_start_events, iti_events)]
            mean_iti_overtime = np.mean(iti_overtime)
        else:
            mean_iti_overtime = []
        session_itis.append(mean_iti_overtime)

    iti_by_session[animal] = session_itis

# Plot histogram of ITI durations > min:
   
# p1 = figure(title="Time over ITI duration (s)",tools="save, zoom_in, zoom_out, pan",
#             background_fill_color="white")

# hist, edges = np.histogram(np.array(iti_overtime)/1E6, density=True, bins=20)
# p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
#         fill_color="#036564", line_color="#033649")
# show(p1)


# Plot ITI for each animal across sessions: 
s1 = figure(title="Time over ITI duration (s)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")
colors = ["red", "orange", "green", "blue"]
for aidx,animal in enumerate(iti_by_session.keys()):
    print animal
#     for sidx,session in enumerate(session_dates):
#         if len(iti_by_session[animal])==0:
#             continue
#         else:
#             s1.circle(sidx, np.array(iti_by_session[animal][sidx])/1E6, color=colors[aidx], size=10, alpha=0.5)

    s1.line(range(len(iti_by_session[animal])), np.array(iti_by_session[animal])/1E6, line_color=colors[aidx], line_width=2, alpha=0.2)
    #s1.line(range(len(iti_by_session[animal])), np.array(iti_by_session[animal])/1E6, line_color="gray", line_width=2, alpha=0.2)

mean_iti_overtime = []
for sidx,session in enumerate(sorted(session_dates)):
    mean_iti_overtime.append(np.mean([iti_by_session[animal][sidx] for animal in iti_by_session.keys() if sidx in range(len(iti_by_session[animal]))]))

s1.line(range(len(mean_iti_overtime)), np.array(mean_iti_overtime)/1E6, line_color="black", line_width=3, alpha=0.5)

show(s1)


AttributeError: 'list' object has no attribute 'time'

In [339]:
iti_by_session['Juno']

[3040391.8793388428,
 5323732.9894459099,
 8637923.0737327188,
 869862.56701030931,
 3506611.6121951221,
 1249073.5,
 518111.40000000002]

In [333]:
np.mean([iti_by_session[animal][sidx] for animal in iti_by_session.keys()]) # if sidx in range(iti_by_session[animal][session])])

TypeError: list indices must be integers, not str

In [28]:
n_attempts = []
for idx,trial in enumerate(data[animal][session]['trials'][0:-1]):
    curr_iti_attempts = [ev.value for ev in iti_attempts if ev.time>trial['time'] and ev.time<data[animal][session]['trials'][idx+1]['time']]
    n_attempts.append(max(curr_iti_attempts))
    

In [32]:
print len(data[animal][session]['trials'])
print len(n_attempts)

516
515


In [511]:
# Check respomnse times:
response_times = []
# for idx,(im,res) in enumerate(zip(test_trials, result_evs)):
#     response_time = (res.time - im['time'])/1E6
#     response_times.append(response_time)
for idx,(im,res) in enumerate(zip(image_events[0:len(result_evs)], result_evs)):
    response_time = (res.time - im.time)/1E6
    response_times.append(response_time)
    
# Plot histogram of response-times:
p1 = figure(title="Response time from stim onset (s)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

hist, edges = np.histogram(response_times, density=True, bins=20)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

In [506]:
# Parse stimulus info for each trial:
print "Found %i image trials." % len(image_events)
print "Found %i stimulus update events across trials." % len(all_trial_events)

# GET TRIAL INFO FOR DB:
trial = dict((i+1, dict()) for i in range(len(image_events)))
image_events = sorted(image_events, key=get_timekey)
trial_events = sorted(all_trial_events, key=get_timekey)
runstart_time = trial_events[0].time
for trialidx,iev in enumerate(sorted(image_events, key=get_timekey)):
    trialnum = trialidx + 1
    blankidx = trialidx*2 + 1
    trial[trialnum]['start_time_ms'] = round(iev.time/1E3)
    #trial[trialnum]['end_time_ms'] = round((trial_events[blankidx].time + session_info['ITI'])/1E3)
    ori = iev.value[1]['rotation']
    sf = iev.value[1]['frequency'] #round(iev.value[1]['frequency'], 2)
    stimname = 'grating-ori-%i-sf-%0.2f' % (ori, sf)
    stimpos = [iev.value[1]['xoffset'], iev.value[1]['yoffset']]
    stimsize = iev.value[1]['height']
    trial[trialnum]['stimuli'] = {'stimulus': stimname, 'position': stimpos, 'scale': stimsize}
    trial[trialnum]['stim_on_times'] = round((iev.time - runstart_time)/1E3)
    trial[trialnum]['stim_off_times'] = round((trial_events[blankidx].time - runstart_time)/1E3)

print "Trial 1 info:"
print trial[2]

Found 507 image trials.
Found 1014 stimulus update events across trials.
Trial 1 info:
{'stim_off_times': 28417.0, 'stimuli': {'stimulus': 'grating-ori-0-sf-0.06', 'position': [0, 0], 'scale': 100}, 'start_time_ms': 3220546.0, 'stim_on_times': 24367.0}


In [507]:
# Check stimulus ON times:
stim_on_durations = [trial[t]['stim_off_times'] - trial[t]['stim_on_times'] for t in trial.keys()]
print "longest: ", [v for v in stim_on_durations].index(max(stim_on_durations))

# Plot histogram of stimulus-ON durations
p1 = figure(title="Stimulus on durations (s)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

hist, edges = np.histogram(np.array(stim_on_durations)/1E3, density=True, bins=20)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

longest:  443


In [25]:
# Determine state of training flags:
pre_reward = df.get_events('flag_pre_reward')
pre_reward = [i for i in pre_reward if i.time>= t_boundary[0] and i.time<=t_boundary[1]]
print np.diff([i.time for i in pre_reward])/1E6

guarantee_reward = df.get_events('flag_guarantee_reward')
guarantee_reward = [i for i in guarantee_reward if i.time>=t_boundary[0] and i.time<=t_boundary[1]]
print np.diff([i.time for i in guarantee_reward])/1E6
print guarantee_reward

NameError: name 'df' is not defined

In [ ]:
# plot trials:
